In [5]:
!gsutil cp -r gs://internal.whitehead.ai/daily-dialog/01-24-2022_09_23_11/outputs/best_model .

Copying gs://internal.whitehead.ai/daily-dialog/01-24-2022_09_23_11/outputs/best_model/config.json...
Copying gs://internal.whitehead.ai/daily-dialog/01-24-2022_09_23_11/outputs/best_model/eval_results.txt...
Copying gs://internal.whitehead.ai/daily-dialog/01-24-2022_09_23_11/outputs/best_model/merges.txt...
Copying gs://internal.whitehead.ai/daily-dialog/01-24-2022_09_23_11/outputs/best_model/model_args.json...
- [4 files][449.2 KiB/449.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://internal.whitehead.ai/daily-dialog/01-24-2022_09_23_11/outputs/best_model/optimizer.pt...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. Th

In [2]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

model = ClassificationModel("deberta", "./best_model")

In [16]:
predictions, raw_outputs = model.predict([["Say what is the meaning of life?", "I dont know"]])

convert_to_label = lambda n: ["__dummy__ (0), inform (1), question (2), directive (3), commissive (4)".split(', ')[i] for i in n]
convert_to_label(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
# Import daily_dialog dataset
from datasets import load_dataset
daily_dialog = load_dataset("daily_dialog")

# Processor
def process_row(batch):    
    result = {}
    text_a = result["text_a"] = []
    text_b = result["text_b"] = []
    labels = result["labels"] = []
    
    acts_batch = batch["act"]
    dialog_batch = batch["dialog"]
    
    for row_idx in range(len(dialog_batch)):
        
        acts = acts_batch[row_idx]
        dialog = dialog_batch[row_idx]
        num = len(dialog)
        assert num == len(acts)

        for idx in range(num):
            text_a.append("" if idx == 0 else dialog[idx-1])
            text_b.append(dialog[idx])
            labels.append(acts[idx])
    
    return result

# Process dataset
mapped = daily_dialog.map(process_row, batched=True, remove_columns=['dialog', 'act', 'emotion'], load_from_cache_file=True).flatten()

# Split model into train, dev etc.
train_df = mapped['train'].to_pandas()
eval_df = mapped['validation'].to_pandas()

Using custom data configuration default
Reusing dataset daily_dialog (/home/jupyter/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c/cache-d4b9dada34f73591.arrow
Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c/cache-7f20f03e2165488c.arrow
Loading cached processed dataset at /home/jupyter/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c/cache-bae8ae2fd66903f5.arrow


In [3]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)

  0%|          | 0/8069 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Running Evaluation:   0%|          | 0/1009 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diwank (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
